In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [3]:
df = pd.read_csv("/kaggle/input/human-activity-recognition/time_series_data_human_activities.csv")
df.head()

,user,activity,timestamp,x-axis,y-axis,z-axis
0,1,Walking,4991922345000,0.69,10.80,-2.03
1,1,Walking,4991972333000,6.85,7.44,-0.50
2,1,Walking,4992022351000,0.93,5.63,-0.50
3,1,Walking,4992072339000,-2.11,5.01,-0.69
4,1,Walking,4992122358000,-4.59,4.29,-1.95


In [4]:
df.isnull().sum()

user         0
activity     0
timestamp    0
x-axis       0
y-axis       0
z-axis       0
dtype: int64

In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
df[['x-axis', 'y-axis', 'z-axis']] = scaler.fit_transform(df[['x-axis', 'y-axis', 'z-axis']])


In [6]:
encoder = LabelEncoder()
df['activity'] = encoder.fit_transform(df['activity'])

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Dense

# Limit the data to the first 50,000 rows
df_subset = df.iloc[:50000]

def create_sequences(df, window_size=100):
    sequences = []
    labels = []
    
    for i in range(len(df) - window_size):
        seq = df[['x-axis', 'y-axis', 'z-axis']].iloc[i:i + window_size].values
        label = df['activity'].iloc[i + window_size]
        sequences.append(seq)
        labels.append(label)
    
    return np.array(sequences), np.array(labels)

window_size = 100

# Create sequences from the subset of data (50,000 rows)
X, y = create_sequences(df_subset, window_size)

# Convert labels to categorical (one-hot encoding)
y = to_categorical(y)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN+LSTM model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))  # Assuming 6 activity classes
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_test, y_test), verbose=1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

# Print the results
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test Loss: {test_loss:.4f}')


Epoch 1/15
624/624 ━━━━━━━━━━━━━━━━━━━━ 24s 35ms/step - accuracy: 0.7879 - loss: 0.6454 - val_accuracy: 0.9082 - val_loss: 0.2586
Epoch 2/15
624/624 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.9199 - loss: 0.2304 - val_accuracy: 0.9454 - val_loss: 0.1673
Epoch 3/15
624/624 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.9494 - loss: 0.1457 - val_accuracy: 0.9581 - val_loss: 0.1204
Epoch 4/15
624/624 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.9637 - loss: 0.1127 - val_accuracy: 0.9795 - val_loss: 0.0662
Epoch 5/15
624/624 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.9721 - loss: 0.0918 - val_accuracy: 0.9842 - val_loss: 0.0527
Epoch 6/15
624/624 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.9788 - loss: 0.0728 - val_accuracy: 0.9841 - val_loss: 0.0492
Epoch 7/15
624/624 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.9809 - loss: 0.0610 - val_accuracy: 0.9917 - val_loss: 0.0330
Epoch 8/15
624/624 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.9860 - loss: 0.0506 - 

In [9]:
model.save('human_activity_recognition_model.h5')